In [3]:
!pip install xlwt

    100% |████████████████████████████████| 102kB 237kB/s a 0:00:01
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/lib/python3.5/dist-packages/xlwt-1.3.0.dist-info'
Consider using the `--user` option or check the permissions.



In [13]:
import sys
from bs4 import BeautifulSoup
import urllib.request
import requests
from urllib.parse import quote

import os
import xlwt

import re

In [14]:
FISRT_URL = "https://www.tripadvisor.com/ShowForum-g294196-i8160-o"
LAST_URL = "-South_Korea.html"
#주소창 앞 뒤로 적기

#url만들기
def get_link(URL, save_file):
    for i in range(3):#i는 0에서 부터 1씩 증가합니다. range(숫자) 번까지 반복합니다.
        current_page_num = 0 + i*20#페이지를 분석하여 숫자가 몇씩 증가하냐에 따라 수정합니다.
        position = URL.index('-o')#url링크의 어느 부분에 숫자가 들어가는지
        URL_with_page_num = URL[: position+2] + str(current_page_num) \
                            + URL[position+2 :]#position+숫자는 몇칸 뒤인지
        print(URL_with_page_num) # 완성된 url이 정상인지 확인하고싶을떄 사용
        importurl = URL_with_page_num
        r = requests.get(importurl)
        soup = BeautifulSoup(r.content.decode('euc-kr','replace'), "lxml")
        #soup = BeautifulSoup(r.content, "lxml")
        for title in soup.find_all('b')[:20]:
            title_link = title.select('a')
            last_URL = title_link[0]['href']
            print(last_URL) #이상없음
            first_URL = "https://www.tripadvisor.com"
            global article_URL
            article_URL = first_URL + last_URL
            print(article_URL)
            get_text(article_URL, save_file)
            

In [15]:
def get_text(URL, save_file):
    global question_post_article
    global answer_post_article
    global post_title
    global post_location
    global data_post
    global post_reply

    url_list = article_URL
    r = requests.get(url_list)
    soup = BeautifulSoup(r.content.decode('euc-kr','replace'), "lxml")

    question_post_article = str(soup.select('div.postBody')[:1])
    question_post__cleaner(question_post_article)
    answer_post_article = str(soup.select('div.postBody')[1:])
    answer_post_cleaner(answer_post_article)

    post_title = str(soup.select('h1')[1:2])
    post_title_cleaner(post_title)
    post_reply = str(soup.select('h5')[1:2])
    post_reply_cleaner(post_reply)
    
    post_location = str(soup.select('li.third')[:1])
    post_location_cleaner(post_location)
    data_post = str(soup.select('div.postDate')[:1])
    data_post_cleaner(data_post)
    #print(data_post_cln)
    print(post_title)

    for i in range(0,1):
        global name_count
        global worksheet
        global workbook
        global doc_name
        name_count = name_count + 1
        worksheet.write(name_count,0, name_count)
        worksheet.write(name_count,1, post_location_cln)
        worksheet.write(name_count,2, post_title_cln)
        worksheet.write(name_count,3, data_post_cln)
        worksheet.write(name_count,4, question_post_cln)
        worksheet.write(name_count,5, answer_post_cln)
        worksheet.write(name_count,6, post_reply_cln)

        workbook.save(os.path.join(os.path.abspath("."), doc_name))
        

In [16]:
def question_post__cleaner(a_text):
    global question_post_article
    global question_post_cln

    text = a_text
    text = re.sub('<script.*?>.*?</script>', '', text, 0, re.I|re.S)
    text = re.sub('<.+?>', '', text, 0, re.I|re.S)
    question_post_cln = re.sub('&nbsp;|\t|\r|\n|\[|\]', '', text)

def answer_post_cleaner(b_text):
    global answer_post_article
    global answer_post_cln

    text = b_text
    text = re.sub('<script.*?>.*?</script>', '', text, 0, re.I|re.S)
    text = re.sub('<.+?>', '', text, 0, re.I|re.S)
    answer_post_cln = re.sub('&nbsp;|\t|\r|\n|\[|\]', '', text)

def post_title_cleaner(c_text):
    global post_title
    global post_title_cln

    text = c_text
    text = re.sub('<script.*?>.*?</script>', '', text, 0, re.I|re.S)
    text = re.sub('<.+?>', '', text, 0, re.I|re.S)
    post_title_cln = re.sub('&nbsp;|\t|\r|\n|\[|\]', '', text)
    
def post_reply_cleaner(f_text):
    global post_reply
    global post_reply_cln

    text = f_text
    text = re.sub('<script.*?>.*?</script>', '', text, 0, re.I|re.S)
    text = re.sub('<.+?>', '', text, 0, re.I|re.S)
    post_reply_cln = re.sub('&nbsp;|\t|\r|\n|\[|\]', '', text)

def post_location_cleaner(d_text):
    global post_location
    global data_post_cln

    text = d_text
    text = re.sub('<script.*?>.*?</script>', '', text, 0, re.I|re.S)
    text = re.sub('<.+?>', '', text, 0, re.I|re.S)
    data_post_cln = re.sub('&nbsp;|\t|\r|\n|\[|\]', '', text)

def data_post_cleaner(e_text):
    global data_post
    global post_location_cln

    text = e_text
    text = re.sub('<script.*?>.*?</script>', '', text, 0, re.I|re.S)
    text = re.sub('<.+?>', '', text, 0, re.I|re.S)
    post_location_cln = re.sub('&nbsp;|\t|\r|\n|\[|\]', '', text)

In [17]:
def main():
    global worksheet
    global workbook
    global name_count
    global doc_name
    name_count = 0
    URL = FISRT_URL  + LAST_URL
    workbook = xlwt.Workbook(encoding='euc-kr')
    workbook.default_style.font.heignt = 20*11   #font설정 11pt

    worksheet = workbook.add_sheet(u'시트0')
    worksheet.write(0,0,u"게시글")
    worksheet.write(0,1,u"날짜")
    worksheet.write(0,2,u"제목")
    worksheet.write(0,3,u"지역")
    worksheet.write(0,4,u"질문")
    worksheet.write(0,5,u"답변")
    worksheet.write(0,6,u"Reply")
    doc_name = "newTrip_KR.csv"

    get_link(URL, doc_name)


if __name__ == '__main__':
    main()

https://www.tripadvisor.com/ShowForum-g294196-i8160-o0-South_Korea.html
/ShowTopic-g294197-i8161-k11543342-Spa_On_Air-Seoul.html
https://www.tripadvisor.com/ShowTopic-g294197-i8161-k11543342-Spa_On_Air-Seoul.html
[<h1 id="HEADING">
Spa On Air
</h1>]
/ShowTopic-g294197-i8161-k11543243-Solo_Trip-Seoul.html
https://www.tripadvisor.com/ShowTopic-g294197-i8161-k11543243-Solo_Trip-Seoul.html
[<h1 id="HEADING">
Solo Trip
</h1>]
/ShowTopic-g294196-i8160-k11536579-Train_to_Busan-South_Korea.html
https://www.tripadvisor.com/ShowTopic-g294196-i8160-k11536579-Train_to_Busan-South_Korea.html
[<h1 id="HEADING">
Train to Busan
</h1>]
/ShowTopic-g297884-i12477-k11534104-Fish_chips_around_Jinju_Namhae-Busan.html
https://www.tripadvisor.com/ShowTopic-g297884-i12477-k11534104-Fish_chips_around_Jinju_Namhae-Busan.html
[<h1 id="HEADING">
fish &amp; chips around Jinju / Namhae
</h1>]
/ShowTopic-g983296-i13236-k11543298-GPS_code_for_Jeju_places_of_interest-Jeju_Island.html
https://www.tripadvisor.com/ShowTop

[<h1 id="HEADING">
21 days South Korea
</h1>]
/ShowTopic-g983296-i13236-k11528174-Bus_from_airport_o_junmoon_family_resort-Jeju_Island.html
https://www.tripadvisor.com/ShowTopic-g983296-i13236-k11528174-Bus_from_airport_o_junmoon_family_resort-Jeju_Island.html
[<h1 id="HEADING">
Bus from airport o junmoon family resort
</h1>]
/ShowTopic-g294196-i8160-k11531896-Is_Jirisan_worth_it_for_only_day_hikes-South_Korea.html
https://www.tripadvisor.com/ShowTopic-g294196-i8160-k11531896-Is_Jirisan_worth_it_for_only_day_hikes-South_Korea.html
[<h1 id="HEADING">
is Jirisan worth it for only day hikes?
</h1>]
/ShowTopic-g294197-i8161-k11524870-How_does_Seoul_subway_charge_your_card-Seoul.html
https://www.tripadvisor.com/ShowTopic-g294197-i8161-k11524870-How_does_Seoul_subway_charge_your_card-Seoul.html
[<h1 id="HEADING">
how does Seoul subway charge your card?
</h1>]
/ShowTopic-g294197-i8161-k11183860-Seoul_South_Korea_Travel_Buddy-Seoul.html
https://www.tripadvisor.com/ShowTopic-g294197-i8161-k1118